In [1]:
import numpy as np

with open('words_250000_train.txt', 'r') as f:
    words = f.read().splitlines()
rng = np.random.default_rng(46)
data = rng.permutation(words)
# data=data[:500]
split_index= int(0.95 * len(data))
train_words = data[:split_index]
val_words = data[-1000:]


In [37]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, Bidirectional, LSTM, Dropout, TimeDistributed, Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.sequence import pad_sequences

class LSTMWordPredictor:
    def __init__(self, weights_path="lstm_model4.h5", max_word_length=20):
        self.chars = list("abcdefghijklmnopqrstuvwxyz0")
        self.char_to_int = {c: i for i, c in enumerate(self.chars)}
        self.int_to_char = {i: c for i, c in enumerate(self.chars)}
        self.vocab_size = len(self.chars)
        self.max_word_length = max_word_length
        self.model = self.build_model()
        self.model.load_weights(weights_path)

    def build_model(self):
        model = Sequential()
        model.add(Embedding(input_dim=self.vocab_size, output_dim=64, trainable=True))
        model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
        model.add(Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=l2(0.001))))
        model.add(Dropout(0.4))
        model.add(TimeDistributed(Dense(self.vocab_size, activation='softmax')))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

#     def predict(self, word_with_missing, guessed_letters):
#         word_with_missing = word_with_missing.replace(' ', '')  # Replace '_' with '0' for consistency
#         word_with_missing = word_with_missing.replace('_', '0')  # Replace '_' with '0' for consistency
#         word_encoded = [self.char_to_int.get(char, self.char_to_int['0']) for char in word_with_missing]
#         word_padded = pad_sequences([word_encoded], maxlen=self.max_word_length, padding='post')
#         prediction = self.model.predict(word_padded, verbose=0)[0]

#         for i, char in enumerate(word_with_missing):
#             if char == '0':
#                 probabilities = prediction[i]
#                 sorted_indices = np.argsort(-probabilities)
#                 pred=[self.int_to_char[idx] for idx in sorted_indices]
#                 return pred
#                 for idx in sorted_indices:
#                     predicted_char = self.int_to_char[idx]
#                     if predicted_char != '0' and predicted_char not in guessed_letters:
#                         return predicted_char
                
#         return None
    def predict(self, word_with_missing, guessed_letters):
        # word_with_missing = word_with_missing.replace(' ', '').replace('_', '0')
        word_encoded = [self.char_to_int.get(char, self.char_to_int['0']) for char in word_with_missing]
        word_padded = pad_sequences([word_encoded], maxlen=self.max_word_length, padding='post')
        prediction = self.model.predict(word_padded, verbose=0)[0]

        best_char = None
        best_prob = -1

        for i, char in enumerate(word_with_missing):
            if char == '0':
                probabilities = prediction[i]
                for idx in np.argsort(-probabilities):
                    predicted_char = self.int_to_char[idx]
                    if predicted_char != '0' and predicted_char not in guessed_letters:
                        prob = probabilities[idx]
                        if prob > best_prob:
                            best_prob = prob
                            best_char = predicted_char
                        break  # Only consider top valid char per position

        return best_char

In [40]:
# words = ["hello", "world", "another", "example"]  # your full word list
predictor = LSTMWordPredictor(weights_path="lstm_model3.h5")
pred = predictor.predict("an00r", guessed_letters={'o','i','s','t','d','a','l','u'})
print(pred)

e


In [2]:
import numpy as np
import torch
from transformers import ByT5Tokenizer, T5Config, T5ForSequenceClassification
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, Bidirectional, LSTM, Dropout, TimeDistributed, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.regularizers import l2

BYT5_MASK_TOKEN = "<extra_id_0>"
BYT5_SEP_TOKEN = "<sep>"

class LSTMWordPredictor:
    def __init__(self, weights_path="lstm_model6.h5", max_word_length=20):
        self.chars = list("abcdefghijklmnopqrstuvwxyz0")
        self.char_to_int = {c: i for i, c in enumerate(self.chars)}
        self.int_to_char = {i: c for i, c in enumerate(self.chars)}
        self.vocab_size = len(self.chars)
        self.max_word_length = max_word_length
        self.model = self.build_model()
        self.model.load_weights(weights_path)

    def build_model(self):
        model = Sequential()
        model.add(Embedding(input_dim=self.vocab_size, output_dim=64, trainable=True))
        model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
        model.add(Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=l2(0.001))))
        model.add(Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=l2(0.001))))
        model.add(Dropout(0.4))
        model.add(TimeDistributed(Dense(self.vocab_size, activation='softmax')))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

    def predict(self, word_with_missing, guessed_letters):
        word_encoded = [self.char_to_int.get(char, self.char_to_int['0']) for char in word_with_missing]
        word_padded = pad_sequences([word_encoded], maxlen=self.max_word_length, padding='post')
        prediction = self.model.predict(word_padded, verbose=0)[0]

        best_char = None
        best_prob = -1

        for i, char in enumerate(word_with_missing):
            if char == '0':
                probabilities = prediction[i]
                for idx in np.argsort(-probabilities):
                    predicted_char = self.int_to_char[idx]
                    if predicted_char != '0' and predicted_char not in guessed_letters:
                        prob = probabilities[idx]
                        if prob > best_prob:
                            best_prob = prob
                            best_char = predicted_char
                        break
        return best_char

class ByT5HangmanPlayer:
    def __init__(self, saved_model_path=None,lstm=None):
        self.tokenizer = ByT5Tokenizer.from_pretrained("google/byt5-small")
        self.config = T5Config.from_pretrained("google/byt5-small")
        self.config.num_labels = 26
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = T5ForSequenceClassification.from_pretrained("byt5-checkpoint-epoch7b", config=self.config).to(self.device)
        self.lstm_model = lstm

    def eval(self):
        self.model.eval()

    def simulate_hangman_transformers(self, word, max_wrong_guesses=6, verbose=1):
        word_idxs = {}
        all_letters = [chr(i) for i in range(ord('a'), ord('z') + 1)]
        for i, c in enumerate(word):
            word_idxs.setdefault(c, []).append(i)

        guesses = {}
        encoded_word = "*" * len(word)
        num_wrong = 0
        self.eval()

        # if verbose:
        #     print(f"[WORD]: {word}")

        while encoded_word != word and num_wrong < max_wrong_guesses:
            missing_count = encoded_word.count("*")

            if missing_count <= 0:
                masked = encoded_word.replace("*", "0")
                guess = self.lstm_model.predict(masked, guessed_letters=set(guesses.keys()))
                print(guess,end=" ")
            else:
                state = ''.join(guesses.keys()) + BYT5_SEP_TOKEN + encoded_word.replace('*', BYT5_MASK_TOKEN)
                enc = self.tokenizer(state, padding="max_length", truncation=True, max_length=64, return_tensors="pt").to(self.device)

                with torch.no_grad():
                    logits = self.model(**enc).logits

                arr = logits.cpu().numpy()[0]
                guess_idx = np.argmax(arr)
                guess = all_letters[guess_idx]

                while guess in guesses:
                    arr[guess_idx] = -np.inf
                    guess_idx = np.argmax(arr)
                    guess = all_letters[guess_idx]

            if guess in word_idxs:
                for i in word_idxs[guess]:
                    encoded_word = encoded_word[:i] + guess + encoded_word[i+1:]
            else:
                num_wrong += 1

            guesses[guess] = True

            if verbose==1:
                print(f"  Guess: {guess.upper():<2} → {encoded_word}  (Wrong guesses: {num_wrong})")

        result = encoded_word == word
        if verbose==2 or verbose==1:
            print(f"Result: {'✅ CORRECT' if result else '❌ FAILED'} | Final: {encoded_word} {guesses.keys()}\n")
        return result
    def test_accuracy(self, words, verbose=1):
        n=len(words)
        count=1
        correct = 0
        for w in words:
            print(f"{(correct / count) * 100:.2f} {count} / {n}", end="\r")
            correct += self.simulate_hangman_transformers(w, verbose=verbose)
            count+=1
        return correct / len(words)
    
# import torch
# import numpy as np
from transformers import CanineTokenizer, CanineConfig, CanineForSequenceClassification
from transformers import CanineConfig, CanineTokenizer, AutoModelForSequenceClassification

# # Constants for separating and masking in the CANINE input sequence
# CANINE_SEP_TOKEN   = " [SEP] "
# CANINE_MASK_TOKEN  = "[MASK]"

class CanineHangmanPlayer:
    def __init__(self, pretrained_model_path: str = "google/canine-s", device: torch.device = None,lstm=None):
        # Load CANINE tokenizer & config
        self.tokenizer = CanineTokenizer.from_pretrained('google/canine-s')
        self.config    = CanineConfig.from_pretrained(pretrained_model_path)
        self.config.num_labels = 26
        self.lstm_model=lstm
        
        # Set up device
        self.device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # Load a sequence-classification head on top of CANINE
        self.model =AutoModelForSequenceClassification.from_pretrained(
            pretrained_model_path,
            config=self.config
        ).to(self.device)
        
        # Tokens for building the game state string
        self.CANINE_MASK_TOKEN = self.tokenizer.mask_token
        self.CANINE_SEP_TOKEN = self.tokenizer.sep_token
        
        # Toggle for self-play finetuning
        self.training = False

    def eval(self):
        self.model.eval()

    def simulate_hangman_transformers(self, word: str, max_wrong_guesses: int = 6, verbose: int = 1):
        """
        Play hangman against the CANINE model.
        If self.training is True, returns (model_logits_seq, true_dist_seq, success_flag).
        Otherwise returns just susccess_flag.
        """
        # Build index map of letters→positions
        word_idxs = {}
        for i, c in enumerate(word):
            word_idxs.setdefault(c, []).append(i)

        all_letters = [chr(i) for i in range(ord('a'), ord('z')+1)]
        guesses      = {}
        encoded_word = "*" * len(word)
        num_wrong= 0

        self.eval()
        if self.training:
            outputs_model = []
            outputs_true  = []

        if verbose:
            print(f"[WORD]: {word}")

        # Main guessing loop
#         while encoded_word != word and wrong_count < max_wrong_guesses:
#             # Build the input string
#             state = (
#                 ''.join(guesses.keys())
#                 + self.CANINE_SEP_TOKEN
#                 + encoded_word.replace("*", self.CANINE_MASK_TOKEN)
#             )
#             enc = self.tokenizer(
#                 state,
#                 padding="max_length",
#                 truncation=True,
#                 max_length=64,
#                 return_tensors="pt"
#             ).to(self.device)

#             # Forward pass
#             with torch.no_grad():
#                 out = self.model(**enc)
#             logits = out.logits             # shape: (1, 26)
#             arr_np = logits[0].cpu().numpy()

#             # If training, build the “true” distribution for this step
#             if self.training:
#                 true_dist = torch.zeros(26, device=self.device)
#                 for ch, positions in word_idxs.items():
#                     if ch not in guesses:
#                         true_dist[ord(ch) - ord('a')] = len(positions)
#                 s = true_dist.sum().item()
#                 if s > 0:
#                     true_dist /= s
#                 else:
#                     raise ValueError("Invalid output distribution (sum=0).")
#                 outputs_model.append(logits)
#                 outputs_true.append(true_dist)

#             # Pick the highest-scoring unseen letter
#             guess_idx = int(np.argmax(arr_np))
#             guess     = all_letters[guess_idx]
#             while guess in guesses:
#                 arr_np[guess_idx] = -np.inf
#                 guess_idx = int(np.argmax(arr_np))
#                 guess     = all_letters[guess_idx]
        while encoded_word != word and num_wrong < max_wrong_guesses:
            missing_count = encoded_word.count("*")

            if missing_count <= 0:
                masked = encoded_word.replace("*", "0")
                guess = self.lstm_model.predict(masked, guessed_letters=set(guesses.keys()))
                print(guess,end=" ")
            else:
                state = ''.join(guesses.keys()) + self.CANINE_SEP_TOKEN + encoded_word.replace('*', self.CANINE_MASK_TOKEN)
                enc = self.tokenizer(state, padding="max_length", truncation=True, max_length=64, return_tensors="pt").to(self.device)

                with torch.no_grad():
                    logits = self.model(**enc).logits

                arr = logits.cpu().numpy()[0]
                guess_idx = np.argmax(arr)
                guess = all_letters[guess_idx]

                while guess in guesses:
                    arr[guess_idx] = -np.inf
                    guess_idx = np.argmax(arr)
                    guess = all_letters[guess_idx]

            # Apply the guess
            if guess in word_idxs:
                for pos in word_idxs[guess]:
                    encoded_word = encoded_word[:pos] + guess + encoded_word[pos+1:]
            else:
                num_wrong += 1

            guesses[guess] = True
            if verbose == 1:
                print(f"  Guess: {guess.upper():<2} → {encoded_word}  (Wrong: {num_wrong})")

        success = (encoded_word == word)
        if verbose == 3:
            print(f"Result: {'✅ CORRECT' if success else '❌ FAILED'} | Final: {encoded_word} {word} {guesses.keys()} \n")

        if self.training:
            return torch.vstack(outputs_model), torch.vstack(outputs_true), success
        return success

#     def simulate_hangman_transformers(self, word, max_wrong_guesses=6, verbose=1):
#         word_idxs = {}
#         all_letters = [chr(i) for i in range(ord('a'), ord('z') + 1)]
#         for i, c in enumerate(word):
#             word_idxs.setdefault(c, []).append(i)

#         guesses = {}
#         encoded_word = "*" * len(word)
#         num_wrong = 0
#         self.eval()

#         if verbose:
#             print(f"[WORD]: {word}")

#         while encoded_word != word and num_wrong < max_wrong_guesses:
#             missing_count = encoded_word.count("*")
#             guessed_letters = set(guesses.keys())

#             guess = None

#             if missing_count > 1:
#                 # Use ByT5 only
#                 state = ''.join(guessed_letters) + BYT5_SEP_TOKEN + encoded_word.replace('*', BYT5_MASK_TOKEN)
#                 enc = self.tokenizer(state, padding="max_length", truncation=True, max_length=64, return_tensors="pt").to(self.device)

#                 with torch.no_grad():
#                     logits = self.model(**enc).logits
#                 probs = torch.nn.functional.softmax(logits, dim=-1).cpu().numpy()[0]

#                 idx = np.argmax(probs)
#                 guess = all_letters[idx]
#                 while guess in guesses:
#                     probs[idx] = -np.inf
#                     idx = np.argmax(probs)
#                     guess = all_letters[idx]

#             else:
#                 # Use both models and compare probabilities
#                 # --- LSTM ---
#                 masked = encoded_word.replace("*", "0")
#                 word_encoded = [self.lstm_model.char_to_int.get(char, self.lstm_model.char_to_int['0']) for char in masked]
#                 word_padded = pad_sequences([word_encoded], maxlen=self.lstm_model.max_word_length, padding='post')
#                 lstm_pred = self.lstm_model.model.predict(word_padded, verbose=0)[0]

#                 lstm_guess, lstm_prob = None, -1
#                 for i, char in enumerate(masked):
#                     if char == '0':
#                         probs = lstm_pred[i]
#                         for idx in np.argsort(-probs):
#                             c = self.lstm_model.int_to_char[idx]
#                             if c != '0' and c not in guessed_letters:
#                                 lstm_guess = c
#                                 lstm_prob = probs[idx]
#                                 break
#                         break

#                 # --- ByT5 ---
#                 state = ''.join(guessed_letters) + BYT5_SEP_TOKEN + encoded_word.replace('*', BYT5_MASK_TOKEN)
#                 enc = self.tokenizer(state, padding="max_length", truncation=True, max_length=64, return_tensors="pt").to(self.device)

#                 with torch.no_grad():
#                     logits = self.model(**enc).logits
#                 probs = torch.nn.functional.softmax(logits, dim=-1).cpu().numpy()[0]

#                 byt5_guess, byt5_prob = None, -1
#                 idx = np.argmax(probs)
#                 byt5_guess = all_letters[idx]
#                 byt5_prob = probs[idx]
#                 while byt5_guess in guesses:
#                     probs[idx] = -np.inf
#                     idx = np.argmax(probs)
#                     byt5_guess = all_letters[idx]
#                     byt5_prob = probs[idx]

#                 # Compare both
#                 guess = lstm_guess if lstm_prob > byt5_prob else byt5_guess

#             if guess in word_idxs:
#                 for i in word_idxs[guess]:
#                     encoded_word = encoded_word[:i] + guess + encoded_word[i+1:]
#             else:
#                 num_wrong += 1

#             guesses[guess] = True

#             if verbose == 1:
#                 print(f"  Guess: {guess.upper():<2} → {encoded_word}  (Wrong guesses: {num_wrong})")

#         result = encoded_word == word
#         if verbose == 2:
#             print(f"Result: {'✅ CORRECT' if result else '❌ FAILED'} | Final: {encoded_word}\n")
#         return result
    def test_accuracy(self, words, verbose=1):
        n=len(words)
        count=1
        correct = 0
        for w in words:
            print(f"{(correct / count) * 100:.2f} {count} / {n}", end="\r")
            correct += self.simulate_hangman_transformers(w, verbose=verbose)
            count+=1
        return correct / len(words)


# if __name__ == "__main__":
#     # val_words = ["melon", "berry", "lemon", "plum", "kiwi"]
#     player = ByT5HangmanPlayer()
#     count=0
#     acc = player.test_accuracy(val_words, verbose=0)
#     print(f"\n📊 Final Gameplay Validation Accuracy: {acc:.4f}")
# if __name__ == "__main__":
#     # A small set of words to validate on
#     # val_words = ["melon", "berry", "lemon", "plum", "kiwi"]
    
#     # Initialize your CANINE-based player
#     player = CanineHangmanPlayer(pretrained_model_path="byt5-checkpoint-epoch5c")
    
#     # (Optional) enable self-play logging
#     player.training = False
#     n=len(val_words)
#     # Run through all validation words
#     correct = 0
#     count=1
#     for word in val_words:
#         print(f"{(correct / count) * 100:.2f} {count} / {n}", end="\r")
#         win = player.simulate_hangman_transformers(word, max_wrong_guesses=6, verbose=1)
#         correct += int(win)
#         count+=1
    
#     # Compute and print accuracy
#     accuracy = correct / len(val_words)
#     print(f"\n📊 Final Gameplay Validation Accuracy: {accuracy:.4f}")


2025-05-15 03:04:06.420985: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-15 03:04:06.421047: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-15 03:04:06.422192: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-15 03:04:06.428384: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-15 03:04:07.248396: W tensorflow/compiler/tf2

In [3]:
# from CanineHangmanPlayer import CanineHangmanPlayer
lstm=LSTMWordPredictor()
model=CanineHangmanPlayer("canine-pretrained-hangman-log/6c",lstm=lstm)
model.training=False

# player = ByT5HangmanPlayer(lstm=lstm)
# player.simulate_hangman_transformers(w,verbose=2)
# print(model.test_accuracy(val_words))
# simulate_hangman_transformers(self, word, max_wrong_guesses=6, verbose=0)

2025-05-15 03:04:08.099445: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-15 03:04:08.105401: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-15 03:04:08.105567: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [ ]:
n=len(val_words)
count=1
correct1 = 0
correct2 = 0
for w in val_words:
    print(f"{(correct1 / count) * 100:.2f} {(correct1 / count) * 100:.2f}  {count} / {n}", end="\r")
    correct1 += model.simulate_hangman_transformers(w, verbose=3)
    # correct2 += player.simulate_hangman_transformers(w,verbose=0)
    count+=1

In [ ]:
import numpy as np
import torch
from transformers import (
    ByT5Tokenizer,
    T5Config,
    T5ForSequenceClassification,
)
from tqdm import tqdm

BYT5_MASK_TOKEN = "<extra_id_0>"
BYT5_SEP_TOKEN = "<sep>"

class ByT5HangmanPlayer:
    def __init__(self, saved_model_path=None):
        self.tokenizer = ByT5Tokenizer.from_pretrained("google/byt5-small")
        self.config = T5Config.from_pretrained("google/byt5-small")
        self.config.num_labels = 26
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # if saved_model_path:
        #     self.model = T5ForSequenceClassification.from_pretrained(saved_model_path, config=self.config).to(self.device)
        # else:
        #     self.model = T5ForSequenceClassification.from_pretrained("google/byt5-small", config=self.config).to(self.device)
        self.model = T5ForSequenceClassification.from_pretrained("byt5-checkpoint-epoch6b", config=self.config).to(self.device)

    def eval(self):
        self.model.eval()

    def simulate_hangman_transformers(self, word, max_wrong_guesses=6, verbose=1):
        word_idxs = {}
        all_letters = [chr(i) for i in range(ord('a'), ord('z') + 1)]
        for i, c in enumerate(word):
            word_idxs.setdefault(c, []).append(i)

        guesses = {}
        encoded_word = "*" * len(word)
        num_wrong = 0
        self.eval()

        if verbose:
            print(f"\n[WORD]: {word}")

        while encoded_word != word and num_wrong < max_wrong_guesses:
            state = ''.join(guesses.keys()) + BYT5_SEP_TOKEN + encoded_word.replace('*', BYT5_MASK_TOKEN)
            enc = self.tokenizer(state, padding="max_length", truncation=True, max_length=64, return_tensors="pt").to(self.device)

            with torch.no_grad():
                logits = self.model(**enc).logits

            arr = logits.cpu().numpy()[0]
            guess_idx = np.argmax(arr)
            guess = all_letters[guess_idx]

            while guess in guesses:
                arr[guess_idx] = -np.inf
                guess_idx = np.argmax(arr)
                guess = all_letters[guess_idx]

            if guess in word_idxs:
                for i in word_idxs[guess]:
                    encoded_word = encoded_word[:i] + guess + encoded_word[i+1:]
            else:
                num_wrong += 1

            guesses[guess] = True

            if verbose:
                print(f"  Guess: {guess.upper():<2} → {encoded_word}  (Wrong guesses: {num_wrong})")

        result = encoded_word == word
        if verbose:
            print(f"Result: {'✅ CORRECT' if result else '❌ FAILED'} | Final: {encoded_word}\n")
        return result

    def test_accuracy(self, words, verbose=1):
        correct = 0
        for w in words:
            correct += self.simulate_hangman_transformers(w, verbose=verbose)
        return correct / len(words)

if __name__ == "__main__":
    import argparse
    import sys

    parser = argparse.ArgumentParser()
    parser.add_argument("--model_path", type=str, default=None, help="Path to pretrained ByT5 model")
    args = parser.parse_args(args=[] if sys.argv[0].endswith("ipykernel_launcher.py") else None)

    # Sample validation words
    # val_words = ["melon", "berry", "lemon", "plum", "kiwi"]

    # Initialize model
    player = ByT5HangmanPlayer(saved_model_path=args.model_path)

    # Run validation with detailed logs
    acc = player.test_accuracy(val_words, verbose=1)
    print(f"\n📊 Final Gameplay Validation Accuracy: {acc:.4f}")